# Strands Agents를 사용한 도구로서의 에이전트


"도구로서의 에이전트"는 AI 시스템의 아키텍처 패턴으로, 전문화된 AI 에이전트를 다른 에이전트가 사용할 수 있는 호출 가능한 함수(도구)로 래핑하는 방식입니다. 이는 다음과 같은 계층적 구조를 만듭니다:

1. 주요 "오케스트레이터" 에이전트가 사용자 상호작용을 처리하고 어떤 전문 에이전트를 호출할지 결정합니다

2. 전문화된 "도구 에이전트"가 오케스트레이터에 의해 호출될 때 도메인별 작업을 수행합니다

이 접근법은 관리자가 전문가들을 조정하는 인간 팀 역학을 모방하며, 각자가 복잡한 문제를 해결하기 위해 고유한 전문 지식을 제공합니다. 단일 에이전트가 모든 것을 처리하려고 하는 대신, 작업이 가장 적절한 전문 에이전트에게 위임됩니다.



## 주요 이점과 핵심 원칙

"도구로서의 에이전트" 패턴은 여러 가지 장점을 제공합니다:

- 관심사의 분리: 각 에이전트가 집중된 책임 영역을 가져 시스템을 이해하고 유지보수하기 쉽게 만듭니다
- 계층적 위임: 오케스트레이터가 어떤 전문가를 호출할지 결정하여 명확한 지휘 체계를 만듭니다
- 모듈식 아키텍처: 전문가들을 전체 시스템에 영향을 주지 않고 독립적으로 추가, 제거 또는 수정할 수 있습니다
- 성능 향상: 각 에이전트가 특정 작업에 최적화된 맞춤형 시스템 프롬프트와 도구를 가질 수 있습니다


In [1]:
!pip install -r requirements.txt

In [2]:
import os

from strands import Agent, tool
from strands_tools import file_write


이 모듈에서는 오케스트레이터 기반 멀티 에이전트 워크플로우를 생성하겠습니다.

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

또한 중첩된 에이전트를 생성할 수 있게 해주는 `use_llm`도 살펴보겠습니다.

## 연구 에이전트

먼저 http_request 도구를 사용하는 기본 연구 어시스턴트를 만들어보겠습니다.

In [3]:
RESEARCH_ASSISTANT_PROMPT = """당신은 전문 연구 어시스턴트입니다. 연구 질문에 대한 응답으로
사실적이고 잘 출처가 명시된 정보만 제공하는 데 집중하세요.
가능할 때마다 항상 출처를 인용하세요."""

In [4]:
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # 여기서 에이전트 AI 검색 도구를 활성화할 수 있습니다
)

query = "Amazon Bedrock과 그 기능에 대한 개요"
# 에이전트를 호출하고 응답을 반환합니다
response = research_agent(query)

# Amazon Bedrock 개요

Amazon Bedrock은 AWS에서 제공하는 완전 관리형 생성형 AI 서비스로, 다양한 기반 모델(FM)을 API를 통해 사용할 수 있게 해주는 플랫폼입니다.

## 주요 기능

### 1. 다양한 기반 모델 제공
Amazon Bedrock은 다음과 같은 여러 AI 모델 제공업체의 기반 모델을 통합 인터페이스로 제공합니다:
- Amazon: Titan 모델 제품군
- Anthropic: Claude 모델 제품군
- AI21 Labs: Jurassic 모델
- Cohere: Command 및 Embed 모델
- Meta: Llama 2 모델
- Stability AI: Stable Diffusion 모델
- Mistral AI: Mistral 모델

[출처: AWS Amazon Bedrock 공식 문서](https://aws.amazon.com/bedrock/)

### 2. 모델 맞춤화 기능
- **모델 미세 조정(Fine-tuning)**: 자체 데이터를 사용하여 기본 모델을 특정 사용 사례에 맞게 조정
- **검색 증강 생성(RAG)**: 자체 데이터로 모델 응답을 강화하여 최신 정보 제공
- **지식 베이스**: 기업 데이터를 검색하고 질문에 답변할 수 있는 RAG 애플리케이션 구축

[출처: AWS Bedrock 기능 페이지](https://aws.amazon.com/bedrock/features/)

### 3. 보안 및 개인 정보 보호
- 데이터와 트래픽은 AWS 내에서 안전하게 암호화
- 고객 데이터는 모델 훈련에 사용되지 않음
- VPC 엔드포인트 지원으로 프라이빗 네트워크 액세스 가능
- AWS IAM과의 통합으로 세밀한 액세스 제어

[출처: AWS Bedrock 보안 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/security.html)

### 4. 애플리케이션 구축 도구
- **Agents for Amazon Bedrock**: 고객 데이터

이제 이 에이전트를 도구로 래핑할 수 있습니다. 다른 에이전트들이 이와 상호작용할 수 있게 됩니다.

#### 도구로서의 에이전트 모범 사례

Strands Agents로 "도구로서의 에이전트" 패턴을 구현할 때:

1. 명확한 도구 문서화: 에이전트의 전문 분야를 설명하는 서술적 docstring 작성
2. 집중된 시스템 프롬프트: 각 전문 에이전트를 해당 도메인에 집중하도록 유지
3. 적절한 응답 처리: 응답을 추출하고 형식화하는 일관된 패턴 사용
4. 도구 선택 가이드: 오케스트레이터에게 각 전문 에이전트를 언제 사용할지에 대한 명확한 기준 제공

In [5]:
@tool
def research_assistant(query: str) -> str:
    """
    연구 관련 쿼리를 처리하고 응답합니다.

    Args:
        query: 사실적 정보가 필요한 연구 질문

    Returns:
        인용이 포함된 상세한 연구 답변
    """
    try:
        # Strands agents를 사용하면 전문 에이전트를 쉽게 생성할 수 있습니다
        research_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # 에이전트를 호출하고 응답을 반환합니다
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"연구 어시스턴트 오류: {str(e)}"

이제 모범 사례를 따라 `product_recommendation_assistant`, `trip_planning_assistant`, 그리고 `orchestrator` 에이전트를 생성해보겠습니다.

### 제품 추천 어시스턴트

In [6]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    적절한 제품을 제안하여 제품 추천 쿼리를 처리합니다.

    Args:
        query: 사용자 선호도가 포함된 제품 문의

    Returns:
        근거가 포함된 개인화된 제품 추천
    """
    try:
        product_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
            system_prompt="""당신은 전문 제품 추천 어시스턴트입니다.
            사용자 선호도를 기반으로 개인화된 제품 제안을 제공하세요. 항상 출처를 인용하세요.""",
        )
        # 에이전트를 호출하고 응답을 반환합니다
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"제품 추천 오류: {str(e)}"

In [7]:
product_recommendation_assistant("플라잉카에 대한 제품 추천")

# 플라잉카 제품 추천

현재 상용화된 진정한 플라잉카는 제한적이지만, 개발 중이거나 예약 주문 가능한 흥미로운 모델들이 있습니다.

## 추천 제품

### 1. PAL-V Liberty
* **특징**: 도로 주행과 비행 모두 가능한 하이브리드 차량
* **가격대**: 약 3억원부터
* **비행 거리**: 약 400-500km
* **참고**: 유럽 도로 안전 인증 완료, 항공 인증 진행 중
* **출처**: PAL-V 공식 웹사이트

### 2. AeroMobil 4.0
* **특징**: 스포츠카 디자인의 변형 날개 구조
* **가격대**: 약 15억원
* **최대 속도**: 도로 160km/h, 비행 360km/h
* **출시 예정**: 제한적 생산 주문 가능
* **출처**: AeroMobil 공식 자료

### 3. Terrafugia Transition
* **특징**: 접이식 날개 디자인, 일반 주유소 연료 사용 가능
* **가격대**: 약 3.5억원
* **비행 범위**: 약 640km
* **출처**: Terrafugia 제품 사양

### 4. Klein Vision AirCar
* **특징**: 2021년 도시 간 비행 테스트 성공
* **변환 시간**: 비행-도로 모드 전환 3분 미만
* **인증**: 슬로바키아 교통국 비행 인증 획득
* **출처**: Klein Vision 발표 자료

## 고려사항
* 대부분의 플라잉카는 조종사 면허 필요
* 각국의 항공 규제 확인 필요
* 현재는 예약 주문 단계이며 실제 배송은 제한적
* 기술과 규제가 빠르게 발전하는 분야

구체적인 구매 계획이 있으시다면 각 제조사의 최신 정보와 귀하의 지역 법규를 확인하는 것이 좋습니다. 추가 정보가 필요하신가요?

'# 플라잉카 제품 추천\n\n현재 상용화된 진정한 플라잉카는 제한적이지만, 개발 중이거나 예약 주문 가능한 흥미로운 모델들이 있습니다.\n\n## 추천 제품\n\n### 1. PAL-V Liberty\n* **특징**: 도로 주행과 비행 모두 가능한 하이브리드 차량\n* **가격대**: 약 3억원부터\n* **비행 거리**: 약 400-500km\n* **참고**: 유럽 도로 안전 인증 완료, 항공 인증 진행 중\n* **출처**: PAL-V 공식 웹사이트\n\n### 2. AeroMobil 4.0\n* **특징**: 스포츠카 디자인의 변형 날개 구조\n* **가격대**: 약 15억원\n* **최대 속도**: 도로 160km/h, 비행 360km/h\n* **출시 예정**: 제한적 생산 주문 가능\n* **출처**: AeroMobil 공식 자료\n\n### 3. Terrafugia Transition\n* **특징**: 접이식 날개 디자인, 일반 주유소 연료 사용 가능\n* **가격대**: 약 3.5억원\n* **비행 범위**: 약 640km\n* **출처**: Terrafugia 제품 사양\n\n### 4. Klein Vision AirCar\n* **특징**: 2021년 도시 간 비행 테스트 성공\n* **변환 시간**: 비행-도로 모드 전환 3분 미만\n* **인증**: 슬로바키아 교통국 비행 인증 획득\n* **출처**: Klein Vision 발표 자료\n\n## 고려사항\n* 대부분의 플라잉카는 조종사 면허 필요\n* 각국의 항공 규제 확인 필요\n* 현재는 예약 주문 단계이며 실제 배송은 제한적\n* 기술과 규제가 빠르게 발전하는 분야\n\n구체적인 구매 계획이 있으시다면 각 제조사의 최신 정보와 귀하의 지역 법규를 확인하는 것이 좋습니다. 추가 정보가 필요하신가요?\n'

### 여행 계획 어시스턴트

In [8]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    여행 일정을 생성하고 여행 조언을 제공합니다.

    Args:
        query: 목적지와 선호도가 포함된 여행 계획 요청

    Returns:
        상세한 여행 일정 또는 여행 조언
    """
    try:
        travel_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
            system_prompt="""당신은 전문 여행 계획 어시스턴트입니다.
            사용자 선호도를 기반으로 상세한 여행 일정을 생성하세요.""",
        )
        # 에이전트를 호출하고 응답을 반환합니다
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"여행 계획 오류: {str(e)}"

### 오케스트레이터 에이전트

In [9]:
# 명확한 도구 선택 가이드가 포함된 오케스트레이터 시스템 프롬프트 정의
MAIN_SYSTEM_PROMPT = """
당신은 쿼리를 전문 에이전트에게 라우팅하는 어시스턴트입니다:
- 연구 질문과 사실적 정보의 경우 → research_assistant 도구 사용
- 제품 추천과 쇼핑 조언의 경우 → product_recommendation_assistant 도구 사용
- 여행 계획과 일정의 경우 → trip_planning_assistant 도구 사용
- 전문 지식이 필요하지 않은 간단한 질문의 경우 → 직접 답변

항상 사용자의 쿼리를 기반으로 가장 적절한 도구를 선택하세요.
"""

In [10]:
# Strands Agents는 에이전트 도구의 쉬운 통합을 허용합니다
orchestrator = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [11]:
# 예시: 전자상거래 고객 서비스 시스템
customer_query = (
    "등산화를 찾고 있습니다. 최종 응답을 현재 디렉토리에 작성해주세요."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# 오케스트레이터가 자동으로 여러 전문 에이전트가 필요하다고 판단합니다
response = orchestrator(customer_query)

제품 추천에 대한 문의를 주셨네요. 등산화에 대한 추천 정보를 찾아드리겠습니다. 추천 결과를 파일로 작성해 드리겠습니다.
Tool #1: product_recommendation_assistant
# 등산화 구매 가이드

안녕하세요! 등산화를 찾고 계시는군요. 좋은 등산화는 등산 경험의 질을 크게 좌우하는 중요한 장비입니다. 

## 좋은 등산화의 특징

1. **접지력과 미끄럼 방지**: 다양한 지형에서 안정적인 접지력을 제공하는 아웃솔 패턴
2. **방수/투습성**: GORE-TEX 같은 방수 멤브레인으로 발을 건조하게 유지
3. **발목 지지력**: 중급 이상 산행에서는 미드컷/하이컷 디자인이 발목 보호에 유리
4. **쿠셔닝**: 장시간 산행에도 편안함을 유지하는 미드솔
5. **내구성**: 마모와 찢김에 강한 소재 사용
6. **무게**: 가벼울수록 피로도 감소
7. **발 형태 적합성**: 개인 발 모양에 맞는 라스트(last) 구조

## 추천 모델

### 초급자용/가벼운 산행
- **살로몬 X 울트라 4 GTX**: 가벼우면서도 안정적인 지지력, 방수 기능 탑재 (출처: Salomon)
- **머렐 모압 3 GTX**: 편안한 착화감과 우수한 접지력, 합리적 가격 (출처: Merrell)

### 중급자용/중장거리 산행
- **라스포르티바 울트라 랩토르 II GTX**: 우수한 쿠셔닝과 안정성, 기술적 지형에 적합 (출처: La Sportiva)
- **스카르파 조디악 GTX**: 뛰어난 발목 지지력과 내구성, 암벽 지형에 강함 (출처: Scarpa)

### 장거리 트레킹/백패킹
- **롤링스 알타 GTX**: 견고한 구조와 뛰어난 내구성, 무거운 배낭 착용 시 안정감 (출처: Lowa)
- **바세치 비올라 GTX**: 전통적인 가죽 소재와 현대적 기술의 조합, 우수한 내구성 (출처: Vasque)

등산화는 반드시 직접 신어보고 본인 발에 맞는 편안한 제품을 선택하는 것이 중요합니다. 어떤 유형의 산행을 계획하고 계신지, 더 구체적인 조언

╔═ File Write Operation ═╗
║                        ║
║ Path: 등산화_추천.md   ║
║ Size: 905 characters   ║
║                        ║
╚════════════════════════╝

╔═════════════ Write Successful ══════════════╗
║ File written successfully to 등산화_추천.md ║
╚═════════════════════════════════════════════╝

등산화 추천 정보를 성공적으로 `등산화_추천.md` 파일에 작성했습니다. 이 파일에는 좋은 등산화의 주요 특징과 산행 난이도별 추천 모델이 포함되어 있습니다.

파일에서 다음 내용을 확인하실 수 있습니다:
- 좋은 등산화가 갖춰야 할 7가지 주요 특징
- 초급자/가벼운 산행에 적합한 추천 모델
- 중급자/중장거리 산행에 적합한 추천 모델
- 장거리 트레킹/백패킹에 적합한 추천 모델

더 구체적인 등산화 추천이 필요하시거나 특정 브랜드, 가격대, 또는 산행 유형에 맞는 추가 정보가 필요하시면 말씀해주세요.

오케스트레이터의 메시지를 살펴보겠습니다. 여기서 에이전트가 하위 에이전트를 도구로 사용하기로 결정한 것을 볼 수 있습니다

In [12]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': '등산화를 찾고 있습니다. 최종 응답을 현재 디렉토리에 작성해주세요.'}]},
 {'role': 'assistant',
  'content': [{'text': '제품 추천에 대한 문의를 주셨네요. 등산화에 대한 추천 정보를 찾아드리겠습니다. 추천 결과를 파일로 작성해 드리겠습니다.'},
   {'toolUse': {'toolUseId': 'tooluse_VLosHGpsSa69GyEywWiMRg',
     'name': 'product_recommendation_assistant',
     'input': {'query': '등산화를 찾고 있습니다. 좋은 등산화의 특징과 추천 모델에 대해 알려주세요.'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_VLosHGpsSa69GyEywWiMRg',
     'status': 'success',
     'content': [{'text': '# 등산화 구매 가이드\n\n안녕하세요! 등산화를 찾고 계시는군요. 좋은 등산화는 등산 경험의 질을 크게 좌우하는 중요한 장비입니다. \n\n## 좋은 등산화의 특징\n\n1. **접지력과 미끄럼 방지**: 다양한 지형에서 안정적인 접지력을 제공하는 아웃솔 패턴\n2. **방수/투습성**: GORE-TEX 같은 방수 멤브레인으로 발을 건조하게 유지\n3. **발목 지지력**: 중급 이상 산행에서는 미드컷/하이컷 디자인이 발목 보호에 유리\n4. **쿠셔닝**: 장시간 산행에도 편안함을 유지하는 미드솔\n5. **내구성**: 마모와 찢김에 강한 소재 사용\n6. **무게**: 가벼울수록 피로도 감소\n7. **발 형태 적합성**: 개인 발 모양에 맞는 라스트(last) 구조\n\n## 추천 모델\n\n### 초급자용/가벼운 산행\n- **살로몬 X 울트라 4 GTX**: 가벼우면서도 안정적인 지지력, 방수 기능 탑재 

In [13]:
customer_query = "파타고니아 여행 계획을 도와주실 수 있나요"

response = orchestrator(customer_query)

파타고니아 여행 계획에 도움을 드리겠습니다. 여행 전문 도구를 활용하여 상세한 정보를 찾아드리겠습니다.
Tool #3: trip_planning_assistant
# 파타고니아 여행 계획 가이드

## 최적의 방문 시기
* **베스트 시즌: 11월~3월(남반구 여름)** - 온화한 기후, 최대 15°C까지 상승
* **숄더 시즌: 10월, 4월** - 관광객이 적고 가격이 저렴하지만 날씨 변동성 있음
* **비수기: 5월~9월** - 추운 겨울철, 일부 명소와 트레일 폐쇄, 대신 고요함과 눈 덮인 풍경

## 파타고니아 주요 명소

### 아르헨티나 측
* **로스 글라시아레스 국립공원**
  - 페리토 모레노 빙하: 세계적으로 유명한 활동성 빙하
  - 피츠로이와 세로 토레 트레킹 코스
* **엘 찰텐**: 트레킹의 중심지, 피츠로이 산 뷰
* **우수아이아**: "세상의 끝" 도시, 비글 해협 크루즈

### 칠레 측
* **토레스 델 파이네 국립공원**
  - W 트레일: 5일 코스 클래식 트레킹
  - O 트레일: 8일 풀 서킷
  - 그레이 빙하와 호수
* **푸에르토 나탈레스**: 토레스 델 파이네 국립공원 관문
* **풍티 아레나스**: 마젤란 해협 옆 주요 도시

## 추천 일정 (14일 기준)

### 일정 1: 칠레-아르헨티나 하이라이트
* **1-2일**: 산티아고/부에노스아이레스 도착, 푼타 아레나스/엘 칼라파테로 이동
* **3-5일**: 토레스 델 파이네 국립공원(W 트레일 하이라이트)
* **6-7일**: 푸에르토 나탈레스에서 휴식, 엘 칼라파테로 이동
* **8-10일**: 페리토 모레노 빙하, 로스 글라시아레스 국립공원 탐험
* **11-12일**: 엘 찰텐에서 피츠로이 트레킹
* **13-14일**: 부에노스아이레스/산티아고로 귀환

### 일정 2: 트레킹 중심
* **1-2일**: 푼타 아레나스/푸에르토 나탈레스 도착
* **3-8일**: 토레스 델 파이네 O 서킷 완주(8일 트레킹)
* **9-10일**: 엘 칼라파테로 이

╔════ File Write Operation ════╗
║                              ║
║ Path: 파타고니아_여행계획.md ║
║ Size: 1611 characters        ║
║                              ║
╚══════════════════════════════╝

╔═════════════════ Write Successful ══════════════════╗
║ File written successfully to 파타고니아_여행계획.md ║
╚═════════════════════════════════════════════════════╝

파타고니아 여행 계획을 `파타고니아_여행계획.md` 파일로 성공적으로 저장했습니다!

이 파일에는 다음과 같은 유용한 정보가 포함되어 있습니다:

1. **최적의 방문 시기** - 남반구에 위치한 파타고니아의 계절별 특징과 최적의 여행 시기
2. **주요 명소** - 아르헨티나와 칠레 양국에 걸친 파타고니아의 핵심 관광지
3. **추천 일정** - 14일 기준의 두 가지 일정 제안:
   - 칠레-아르헨티나 하이라이트 여행
   - 트레킹 중심의 일정
4. **필수 준비물** - 의류, 신발, 장비 등 파타고니아 여행에 필요한 준비물 리스트
5. **여행 팁** - 예약, 현금 준비, 날씨 대비, 국경 이동 등 실용적인 조언

파타고니아 여행에 대해 더 구체적인 정보가 필요하시거나 특정 지역, 트레킹 코스, 또는 예산 계획에 대한 질문이 있으시면 언제든지 말씀해 주세요!

In [14]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': '등산화를 찾고 있습니다. 최종 응답을 현재 디렉토리에 작성해주세요.'}]},
 {'role': 'assistant',
  'content': [{'text': '제품 추천에 대한 문의를 주셨네요. 등산화에 대한 추천 정보를 찾아드리겠습니다. 추천 결과를 파일로 작성해 드리겠습니다.'},
   {'toolUse': {'toolUseId': 'tooluse_VLosHGpsSa69GyEywWiMRg',
     'name': 'product_recommendation_assistant',
     'input': {'query': '등산화를 찾고 있습니다. 좋은 등산화의 특징과 추천 모델에 대해 알려주세요.'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_VLosHGpsSa69GyEywWiMRg',
     'status': 'success',
     'content': [{'text': '# 등산화 구매 가이드\n\n안녕하세요! 등산화를 찾고 계시는군요. 좋은 등산화는 등산 경험의 질을 크게 좌우하는 중요한 장비입니다. \n\n## 좋은 등산화의 특징\n\n1. **접지력과 미끄럼 방지**: 다양한 지형에서 안정적인 접지력을 제공하는 아웃솔 패턴\n2. **방수/투습성**: GORE-TEX 같은 방수 멤브레인으로 발을 건조하게 유지\n3. **발목 지지력**: 중급 이상 산행에서는 미드컷/하이컷 디자인이 발목 보호에 유리\n4. **쿠셔닝**: 장시간 산행에도 편안함을 유지하는 미드솔\n5. **내구성**: 마모와 찢김에 강한 소재 사용\n6. **무게**: 가벼울수록 피로도 감소\n7. **발 형태 적합성**: 개인 발 모양에 맞는 라스트(last) 구조\n\n## 추천 모델\n\n### 초급자용/가벼운 산행\n- **살로몬 X 울트라 4 GTX**: 가벼우면서도 안정적인 지지력, 방수 기능 탑재 

### 여러 에이전트 호출

In [15]:
orchestrator.messages = []

In [16]:
query = "스페인에 대한 연구를 해주실 수 있나요? 또한 7일 여행 계획도 도와주세요."

orchestrator(query)

스페인에 관한 연구와 7일 여행 계획을 도와드릴 수 있습니다. 먼저 스페인에 대한 연구부터 진행한 후 여행 계획을 안내해 드리겠습니다.
Tool #5: research_assistant
# 스페인의 역사, 문화, 지리 및 주요 관광지

## 역사

스페인의 역사는 다양한 문명의 영향으로 풍부하고 복잡합니다.

1. **초기 역사**: 이베리아 반도는 페니키아인, 그리스인, 카르타고인이 정착했으며, 기원전 218년부터 로마 지배를 받았습니다[^1].

2. **이슬람 시대(711-1492)**: 무어인들이 이베리아 반도를 정복하여 알-안달루스를 설립했으며, 코르도바를 중심으로 과학, 예술, 건축이 발전했습니다[^2].

3. **레콘키스타(722-1492)**: 기독교 왕국들이 점진적으로 무어인들로부터 영토를 되찾는 과정으로, 1492년 그라나다 정복으로 완성되었습니다[^1].

4. **제국 시대(15-17세기)**: 페르디난드와 이사벨라의 통일 이후, 스페인은 신대륙 탐험으로 글로벌 제국으로 성장했습니다[^3].

5. **현대 스페인**: 스페인 내전(1936-1939) 후 프란시스코 프랑코의 독재를 거쳐, 1975년 이후 민주주의 체제로 전환되었습니다[^1].

## 문화

스페인 문화는 다양한 지역 전통과 역사적 영향이 혼합된 모자이크입니다.

1. **언어**: 공식 언어인 스페인어(카스티야어) 외에도 카탈루냐어, 바스크어, 갈리시아어 등 지역 언어가 존재합니다[^4].

2. **예술**: 피카소, 달리, 고야, 벨라스케스와 같은 세계적인 화가들을 배출했습니다[^3].

3. **건축**: 가우디의 작품들, 이슬람 영향의 알람브라 궁전, 고딕 및 르네상스 양식의 성당들이 특징적입니다[^3].

4. **음식**: 타파스, 파에야, 하몽, 가스파초 등 지중해식 요리가 유명합니다[^5].

5. **축제**: 라 토마티나(토마토 축제), 산 페르민(투우 달리기), 세마나 산타(성 주간) 등 다양한 지역 축제가 있습니다[^5].

6. **플라멩코**: 

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '위의 정보를 바탕으로 스페인에 대한 간략한 요약과 여행 계획을 정리해 드리겠습니다.\n\n## 스페인 개요\n\n스페인은 풍부한 역사와 다양한 문화적 유산을 지닌 나라입니다. 이베리아 반도에 위치하며 로마 제국, 이슬람 시대, 레콘키스타, 제국 시대를 거쳐 현대 민주주의 국가로 발전했습니다. 스페인어(카스티야어)가 공식 언어이며, 카탈루냐어, 바스크어 등 지역 언어도 사용됩니다.\n\n문화적으로는 피카소, 달리, 고야와 같은 세계적 화가들과 가우디의 건축물, 그리고 플라멩코, 타파스, 파에야 같은 독특한 음식문화가 유명합니다. 라 토마티나(토마토 축제), 산 페르민(투우 달리기) 등 다양한 축제도 스페인 문화의 중요한 부분입니다.\n\n지리적으로는 중앙 고원, 피레네 산맥, 다양한 해안선을 갖추고 있으며, 바르셀로나, 마드리드, 세비야, 그라나다, 발렌시아 등 매력적인 도시들이 있습니다.\n\n## 7일 여행 계획 요약\n\n제안된 7일 여행 계획은 스페인의 주요 명소를 효율적으로 둘러볼 수 있도록 구성되었습니다:\n\n- **1-3일차**: 바르셀로나에서 람블라스 거리, 고딕 지구, 사그라다 파밀리아, 구엘 공원 등 가우디 명소 탐방\n- **3-6일차**: 마드리드로 이동하여 프라도 미술관, 레티로 공원, 왕궁, 마요르 광장 등 방문\n- **5일차**: 톨레도 당일 여행 (마드리드에서 기차로 33분)\n- **6일차**: 마드리드에서 추가 문화 체험 및 플라멩코 쇼\n- **7일차**: 세고비아 방문 후 귀국\n\n현지 교통은 도시 간 이동은 AVE 고속철도, 도시 내 이동은 메트로와 도보를 활용하면 편리합니다. 주요 관광지는 사전 예약이 필수이며, 스페인의 늦은 식사 시간(점심 14-16시, 저녁 21-23시)에 맞춰 일정을 조정하는 것이 좋습니다.\n\n이 여행을 통해 스페인의 다양한 역사, 문화

내부적으로 오케스트레이터는:
1. 먼저 `research_assistant`를 호출합니다
2. 그다음 `trip_planning_assistant`를 호출합니다
3. 이러한 전문화된 응답들을 두 쿼리 모두를 다루는 일관된 답변으로 결합합니다

### 순차적 에이전트 통신 패턴


에이전트 도구는 여러 에이전트를 함께 결합할 수도 있습니다. 이 예시에서는 `research_agent`의 출력을 `summary_agent`에 제공하고 요약된 응답을 반환하겠습니다.

In [17]:
 # 사용자 쿼리 정의
topic = "생성형 AI"
# 연구 에이전트 생성
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# 요약 에이전트 생성
summary_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""
    당신은 복잡한 정보를 명확하고 간결한 요약으로 정제하는 데 집중하는 요약 전문가입니다.
    주요 목표는 상세한 정보를 받아 핵심 포인트, 주요 논점, 중요한 데이터를 추출하는 것입니다.
    원본 내용의 정확성을 유지하면서 더 소화하기 쉽게 만들어야 합니다.
    명확성, 간결성, 그리고 정보의 가장 중요한 측면을 강조하는 데 집중하세요.
    """,
)

print("여러 에이전트가 성공적으로 생성되었습니다!")
print(f"\n🔍 연구 에이전트가 작업 중: {topic}\n") 
try:
    # 에이전트 1: 연구 에이전트 호출
    research_response = research_agent(
        f"{topic}에 대한 포괄적인 정보를 수집해주세요."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ 요약 에이전트가 연구를 정제하는 중\n")
    
    # 에이전트 2: 요약 에이전트에게 간결한 요약 생성 요청
    summary_response = summary_agent(
        f"이 연구의 간결한 요약을 작성해주세요: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"연구 어시스턴트 오류: {str(e)}")

여러 에이전트가 성공적으로 생성되었습니다!

🔍 연구 에이전트가 작업 중: 생성형 AI

# 생성형 AI에 대한 포괄적인 정보

생성형 AI(Generative AI)는 콘텐츠를 생성하기 위한 인공지능 시스템으로, 최근 몇 년간 기술적 발전과 함께 다양한 분야에서 주목받고 있습니다. 아래에서 생성형 AI에 대한 포괄적인 정보를 제시합니다.

## 정의 및 작동 원리

생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠(텍스트, 이미지, 음악, 영상 등)를 생성하는 AI 시스템입니다. 주로 다음과 같은 모델 유형을 기반으로 합니다:

- **생성적 적대 신경망(GANs)**: 생성자와 판별자 네트워크 간의 경쟁을 통해 학습하는 구조
- **변분 오토인코더(VAEs)**: 확률적 접근 방식으로 데이터의 잠재 표현을 학습
- **확산 모델(Diffusion Models)**: 점진적으로 노이즈를 제거하며 콘텐츠를 생성
- **트랜스포머(Transformer)**: 자연어 처리와 멀티모달 태스크에서 주로 사용되는 아키텍처

최근에는 대규모 언어 모델(LLMs)이 주목받고 있으며, 대표적으로 GPT(Generative Pre-trained Transformer) 계열이 있습니다[1].

## 주요 적용 분야

생성형 AI는 다양한 분야에서 활용되고 있습니다:

1. **텍스트 생성**: 콘텐츠 작성, 번역, 요약, 챗봇 등
2. **이미지 생성**: DALL-E, Midjourney, Stable Diffusion 등을 통한 사실적 이미지 생성
3. **음악 및 오디오 생성**: 작곡, 음성 합성, 사운드 디자인
4. **비디오 생성**: 짧은 클립부터 장면 합성까지
5. **코드 생성**: GitHub Copilot 등 프로그래밍 지원 도구
6. **제품 설계 및 개발**: 제품 디자인, 구조 최적화 등[2]

## 기술적 발전과 주요 모델

최근 몇 년간 생성형 AI 분야에서 중요한 발전이 있었습니다:

- **2017년**: NVIDIA의 고해상도 GAN 이미지 

## 축하합니다!

Strands Agents에서 에이전트를 도구로 사용하여 더 복잡한 에이전트 애플리케이션을 생성하는 방법을 배웠습니다